In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import imblearn



In [7]:
data_df = pd.read_csv("data01.csv")
data_df.head()

,group,ID,outcome,age,gendera,BMI,hypertensive,atrialfibrillation,CHD with no MI,diabetes,...,Blood sodium,Blood calcium,Chloride,Anion gap,Magnesium ion,PH,Bicarbonate,Lactic acid,PCO2,EF
0,1,125047,0.0,72,1,37.588179,0,0,0,1,...,138.750000,7.463636,109.166667,13.166667,2.618182,7.230,21.166667,0.5,40.0,55
1,1,139812,0.0,75,2,NaN,0,0,0,0,...,138.888889,8.162500,98.444444,11.444444,1.887500,7.225,33.444444,0.5,78.0,55
2,1,109787,0.0,83,2,26.572634,0,0,0,0,...,140.714286,8.266667,105.857143,10.000000,2.157143,7.268,30.571429,0.5,71.5,35
3,1,130587,0.0,43,2,83.264629,0,0,0,0,...,138.500000,9.476923,92.071429,12.357143,1.942857,7.370,38.571429,0.6,75.0,55
4,1,138290,0.0,75,2,31.824842,1,0,0,0,...,136.666667,8.733333,104.500000,15.166667,1.650000,7.250,22.000000,0.6,50.0,55


In [4]:
for col in data_df.columns:
    print(col)

group
ID
outcome
age
gendera
BMI
hypertensive
atrialfibrillation
CHD with no MI
diabetes
deficiencyanemias
depression
Hyperlipemia
Renal failure
COPD
heart rate
Systolic blood pressure
Diastolic blood pressure
Respiratory rate
temperature
SP O2
Urine output
hematocrit
RBC
MCH
MCHC
MCV
RDW
Leucocyte
Platelets
Neutrophils
Basophils
Lymphocyte
PT
INR
NT-proBNP
Creatine kinase
Creatinine
Urea nitrogen
glucose
Blood potassium
Blood sodium
Blood calcium
Chloride
Anion gap
Magnesium ion
PH
Bicarbonate
Lactic acid
PCO2
EF


In [11]:
data_df.rename(columns = {'CHD with no MI':'CHD', 'deficiencyanemias':'anemia', 'atrialfibrillation':'afib'}, inplace = True)

In [15]:
data_df.rename(columns = {'Renal failure':'renalfailure', 'Hyperlipemia': 'hyperlipidemia', 'hypertensive':'htn'}, inplace = True)

In [16]:
for col in data_df.columns:
    print(col)

group
ID
outcome
age
gendera
BMI
htn
afib
CHD
diabetes
anemia
depression
hyperlipidemia
renalfailure
COPD
heart rate
Systolic blood pressure
Diastolic blood pressure
Respiratory rate
temperature
SP O2
Urine output
hematocrit
RBC
MCH
MCHC
MCV
RDW
Leucocyte
Platelets
Neutrophils
Basophils
Lymphocyte
PT
INR
NT-proBNP
Creatine kinase
Creatinine
Urea nitrogen
glucose
Blood potassium
Blood sodium
Blood calcium
Chloride
Anion gap
Magnesium ion
PH
Bicarbonate
Lactic acid
PCO2
EF


In [43]:
comorbid_df=data_df[['htn', 'afib', 'CHD', 'diabetes', 'anemia', 'depression', 'hyperlipidemia', 'renalfailure', 'COPD']]

In [44]:
comorbid_df.head()

,htn,afib,CHD,diabetes,anemia,depression,hyperlipidemia,renalfailure,COPD
0,0,0,0,1,1,0,1,1,0
1,0,0,0,0,1,0,0,0,1
2,0,0,0,0,1,0,0,1,0
3,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,1


In [47]:

comorbid_df['comorbidities'] = comorbid_df.sum(axis=1)

C:\Users\Range\AppData\Local\Temp/ipykernel_12808/853498199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comorbid_df['comorbidities'] = comorbid_df.sum(axis=1)


In [48]:
comorbid_df.head()

,htn,afib,CHD,diabetes,anemia,depression,hyperlipidemia,renalfailure,COPD,comorbidities
0,0,0,0,1,1,0,1,1,0,4
1,0,0,0,0,1,0,0,0,1,2
2,0,0,0,0,1,0,0,1,0,2
3,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,1,4


In [22]:
data_df.head()

,group,ID,outcome,age,gendera,BMI,htn,afib,CHD,diabetes,...,Blood sodium,Blood calcium,Chloride,Anion gap,Magnesium ion,PH,Bicarbonate,Lactic acid,PCO2,EF
0,1,125047,0.0,72,1,37.588179,0,0,0,1,...,138.750000,7.463636,109.166667,13.166667,2.618182,7.230,21.166667,0.5,40.0,55
1,1,139812,0.0,75,2,NaN,0,0,0,0,...,138.888889,8.162500,98.444444,11.444444,1.887500,7.225,33.444444,0.5,78.0,55
2,1,109787,0.0,83,2,26.572634,0,0,0,0,...,140.714286,8.266667,105.857143,10.000000,2.157143,7.268,30.571429,0.5,71.5,35
3,1,130587,0.0,43,2,83.264629,0,0,0,0,...,138.500000,9.476923,92.071429,12.357143,1.942857,7.370,38.571429,0.6,75.0,55
4,1,138290,0.0,75,2,31.824842,1,0,0,0,...,136.666667,8.733333,104.500000,15.166667,1.650000,7.250,22.000000,0.6,50.0,55


In [24]:
new_df=data_df.copy()

In [63]:
new_comorbid_df=comorbid_df.copy()

In [64]:
new_comorbid_df.head()

,htn,afib,CHD,diabetes,anemia,depression,hyperlipidemia,renalfailure,COPD,comorbidities
0,0,0,0,1,1,0,1,1,0,4
1,0,0,0,0,1,0,0,0,1,2
2,0,0,0,0,1,0,0,1,0,2
3,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,1,4


In [71]:
new_df.head()

,group,ID,outcome,age,gendera,BMI,htn,afib,CHD,diabetes,...,Blood sodium,Blood calcium,Chloride,Anion gap,Magnesium ion,PH,Bicarbonate,Lactic acid,PCO2,EF
0,1,125047,0.0,72,1,37.588179,0,0,0,1,...,138.750000,7.463636,109.166667,13.166667,2.618182,7.230,21.166667,0.5,40.0,55
1,1,139812,0.0,75,2,NaN,0,0,0,0,...,138.888889,8.162500,98.444444,11.444444,1.887500,7.225,33.444444,0.5,78.0,55
2,1,109787,0.0,83,2,26.572634,0,0,0,0,...,140.714286,8.266667,105.857143,10.000000,2.157143,7.268,30.571429,0.5,71.5,35
3,1,130587,0.0,43,2,83.264629,0,0,0,0,...,138.500000,9.476923,92.071429,12.357143,1.942857,7.370,38.571429,0.6,75.0,55
4,1,138290,0.0,75,2,31.824842,1,0,0,0,...,136.666667,8.733333,104.500000,15.166667,1.650000,7.250,22.000000,0.6,50.0,55


In [27]:
comorbid_df.to_csv('comorbidities.csv')  